In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
orig_df = pd.read_csv('labeled_data_100k.csv')
orig_df.head()

,from,to,subject,body,is_useful
0,phillip.allen@enron.com,tim.belden@enron.com,NaN,Here is our forecast,0
1,phillip.allen@enron.com,john.lavorato@enron.com,Re,Traveling to have a business meeting takes the...,0
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re,test successful. way to go!!!,0
3,phillip.allen@enron.com,randall.gay@enron.com,NaN,"Randy,Can you send me a schedule of the salary...",1
4,phillip.allen@enron.com,greg.piper@enron.com,Re,NaN,0


In [3]:
df = orig_df.rename(columns={"body": "Body", "is_useful": "Useful"}, errors="raise")
df = df[['Body', 'Useful']]
df['category_id'] = df['Useful']
df['Body'] = df['Body'].fillna("no body")
df.head()

,Body,Useful,category_id
0,Here is our forecast,0,0
1,Traveling to have a business meeting takes the...,0,0
2,test successful. way to go!!!,0,0
3,"Randy,Can you send me a schedule of the salary...",1,1
4,no body,0,0


In [4]:
from io import StringIO

def set_status(key):
    if key == 1:
        return "Useful"
    elif key == 0:
        return "Not-Useful"

col = ['Useful', 'Body']
df = df[col]
df = df[pd.notnull(df['Body'])]
df.columns = ['Useful', 'Body']
df['category_id'] = df['Useful']
df['Useful'] = df['category_id'].apply(set_status)
category_id_df = df[['Useful', 'category_id']].sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Useful']].values)
df = df.head(3000)

<h2>Using Linear Regression SVC</h2>

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder

In [6]:
labels = df['Useful']
text = df['Body']

X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state=0, test_size=0.3)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_transformed = tf_transformer.transform(X_train_counts)
print("Hello\n", tf_transformer)
X_test_counts = count_vect.transform(X_test)
X_test_transformed = tf_transformer.transform(X_test_counts)

labels = LabelEncoder()
y_train_labels_fit = labels.fit(y_train)
y_train_lables_trf = labels.transform(y_train)

def fitted():
    return count_vect, tf_transformer
print(fitted)

Hello
 TfidfTransformer()
<function fitted at 0x7fb4ca811ca0>


In [7]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
import pickle

linear_svc = LinearSVC()
clf = linear_svc.fit(X_train_transformed,y_train_lables_trf)

calibrated_svc = CalibratedClassifierCV(base_estimator=linear_svc,
                                        cv="prefit")

calibrated_svc.fit(X_train_transformed,y_train_lables_trf)

filename = 'usefulness-classify-model.pkl'
with open(filename, 'wb') as file:  
    pickle.dump(linear_svc, file)

predicted = calibrated_svc.predict(X_test_transformed)    # predict on X_test
    
print('Average accuracy on test set={}'.format(np.mean(predicted == labels.transform(y_test))))

Average accuracy on test set=0.9677777777777777
